<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
# Import some necessary modules
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import pandas as pd
import time
import scipy as sp
import pickle
%load_ext autoreload
%autoreload 2

In [3]:
# Import MuSiCal
import musical

# Codes -- Overview

In [4]:
# Read in result after de novo signature discovery. 
with open('./data/validation_internal/Breast-AdenoCA.96.normalizeX_True.nmf.model.pkl', 'rb') as f:
    model_old = pickle.load(f)
# Read in the original data matrix X as pd.DataFrame. This is mostly used for the sample names.
# !!!!!!!!!!!!!!!!!! Make sure that this X matrix precisely matches model_old.X !!!!!!!!!!!!!!!!!!
X = pd.read_csv('./data/validation_internal/Breast-AdenoCA.96.X.csv', index_col=0)
X.head()

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
Type,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,35,36,217,50,35,43,30,22,52,92,...,47,198,71,56,188,193,70,144,162,316
A[C>A]C,29,36,181,20,32,43,27,9,44,48,...,30,158,44,33,169,160,54,111,116,393
A[C>A]G,5,2,36,5,3,7,9,6,8,9,...,7,24,8,3,21,24,9,19,25,48
A[C>A]T,27,22,231,24,24,38,10,12,27,40,...,23,159,47,22,161,184,45,94,139,310
C[C>A]A,25,37,190,33,23,35,15,16,23,86,...,27,134,47,41,166,157,65,90,144,343


In [5]:
# I made some updates on the de novo discovery part of DenovoSig to use pd.DataFrame more, so that
# sample names and signature names are kept. 
# To accomodate that update, the following lines are required to modify old DenovoSig objects. 
# !!!!!!!!!!!!!!!!!! If de novo discovery is performed with new code, no need to do these lines !!!!!!!!!!!!!!!!!!
import copy
model = copy.deepcopy(model_old)
model.X_df = X.astype(float)
model.samples = model.X_df.columns.values
model.features = model.X_df.index.values
model.W_df = pd.DataFrame(model.W, columns=['Sig' + str(i) for i in range(1, model.n_components + 1)],
                          index=model.features)
model.signatures = model.W_df.columns.values
model.H_df = pd.DataFrame(model.H, columns=model.samples, index=model.signatures)

In [6]:
# Read a catalog
# !!!!!!!!!!!!!!!!!! Use an appropriate catalog instead !!!!!!!!!!!!!!!!!!
W_catalog = musical.load_catalog().W

## 2d grid search 

In [8]:
# Define a log-spaced grid. 
# !!!!!!!!!!!!!!!!!! Not sure if this is a good grid. Just as an example !!!!!!!!!!!!!!!!!!
thresh_match_grid = np.geomspace(0.0001, 5, 10)
thresh_refit_grid = np.geomspace(0.0001, 5, 10)
print(thresh_match_grid)

[1.00000000e-04 3.32742119e-04 1.10717318e-03 3.68403150e-03
 1.22583245e-02 4.07886087e-02 1.35720881e-01 4.51600535e-01
 1.50266519e+00 5.00000000e+00]


In [ ]:
# Perform assignment
# Now there is only one thresh for matching and one thresh for refitting. 
# "thresh1" and "thresh2" are now hidden and not accessible to users. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=thresh_match_grid,
    thresh_refit_grid=thresh_refit_grid,
    thresh_new_sig=0.0, # Perhaps set this to 0 to suppress calling new signatures? We only care about sig assignment now anyway.
    indices_associated_sigs=None
)

In [ ]:
# Perform validation
model.validate_grid(
    validate_n_replicates=1, # This is the number of repetitions for each grid point. Previously this was 3. 
    W_cos_dist_thresh=0.02 # We consider grid points with W distance <= min + 0.02 as good. 
)
# Note that now the distance between W_data and W_simul is the MEAN cos distance between the signatures. 
# So no need to muliply 0.02 by the number of signatures. 

## Separate 1d grid searches for matching and refitting 

In [ ]:
# First perform 1d grid search for matching. 
# Refitting threshold will be set to a small value. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=np.geomspace(0.0001, 5, 10),
    thresh_refit_grid=np.array([0.0001]),
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)
model.validate_grid(
    validate_n_replicates=1, 
    W_cos_dist_thresh=0.02
)

In [ ]:
# Look at the best matching parameter
best_thresh_match = model.thresh_match

In [ ]:
# Then perform 1d grid search for refitting.
# Matching parameter will be set to the best one above.
# !!!!!!!!!!!!!!!!!!!! Note that if you run assign_grid() and validate_grid() again, the previous result will be lost
# !!!!!!!!!!!!!!!!!!!! So we should save the first grid search result, and then do the second grid search. 
# !!!!!!!!!!!!!!!!!!!! Or use copy.deepcopy() to copy the model to model2, and then do the second grid search. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=np.array([best_thresh_match]),
    thresh_refit_grid=np.geomspace(0.0001, 5, 10),
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)
model.validate_grid(
    validate_n_replicates=1, 
    W_cos_dist_thresh=0.02
)

## Validate a single signature assignment 

In [ ]:
# We can perform a single assignment using model.assign()
# For example:
model.assign(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match=0.01, 
    thresh_refit=0.02, 
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)

In [ ]:
# We can then perform a single validation for this assignment
model.validate(
    W_s=None,
    H_s=None,
    validate_n_replicates=1
)
# Setting W_s and H_s to None will allow it to use the existing W_s and H_s calculated by model.assign()

## Validate external signature assignments 

In [ ]:
# If we just want to validate an external signature assignment result, also use model.validate().
# For example, if W_s_PCAWG and H_s_PCAWG are PCAWG assignments:
model.validate(
    W_s=W_s_PCAWG,
    H_s=H_s_PCAWG,
    validate_n_replicates=1
)
# !!!!!!!!!!!!! Note that W_s_PCAWG and H_s_PCAWG must be pd.DataFrame !!!!!!!!!!!!!
# The sample names in H_s_PCAWG must match with sample names in model.X_df

# Example

Here we do some mock runs to demonstrate the output. 

In order to save time, we have chosen to use a nmf model. And we further simplify it by running less iterations. 

In [7]:
model.max_iter = 10000
model.min_iter = 2000
model.tol = 1e-7
model.n_replicates = 1
model.ncpu = 1

Let's do a 1d search for matching first. 

In [8]:
model.assign_grid(W_catalog, 
                  thresh_match_grid=np.array([0.001, 0.01, 0.1, 1, 5]), 
                  thresh_refit_grid=np.array([0.0001]), 
                  thresh_new_sig=0.0)

In [10]:
model.validate_grid()

Extracting signatures for n_components = 9..................
n_components = 9, replicate 0 finished.
Time elapsed: 2.61 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.69 seconds.


...

/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.87 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


In [11]:
# Best parameter:
print(model.best_grid_point)
print(model.thresh_match)
print(model.thresh_refit)

(0.1, 0.0001)
0.1
0.0001


In [12]:
# Assigned signatures
print(model.sigs_assigned)

['SBS1' 'SBS2' 'SBS3' 'SBS5' 'SBS8' 'SBS10c' 'SBS13' 'SBS17a' 'SBS17b'
 'SBS18' 'SBS30' 'SBS39' 'SBS44' 'SBS98']


In [13]:
# Number of assigned signatures on the grid:
model.n_sigs_assigned_grid

{(0.001, 0.0001): 49,
 (0.01, 0.0001): 27,
 (0.1, 0.0001): 14,
 (1.0, 0.0001): 8,
 (5.0, 0.0001): 8}

In [14]:
# W distance on the grid:
# Note that the grid point (5, 0.0001) is skipped because it is redundant with (1, 0.0001). 
# So that grid point information will not be shown. 
model.W_cos_dist_mean_grid

{(0.001, 0.0001): 0.0551210687300413,
 (0.01, 0.0001): 0.047527603961973676,
 (0.1, 0.0001): 0.06326554678757877,
 (1.0, 0.0001): 0.18800502197422686}

In [15]:
# H distance on the grid:
model.H_frobenius_dist_mean_grid

{(0.001, 0.0001): 14844.976849639308,
 (0.01, 0.0001): 27478.949346895282,
 (0.1, 0.0001): 12971.836696845217,
 (1.0, 0.0001): 25009.307537158762}

In [17]:
# You can access the matrices as follows:
# model.W_simul_grid is a dictionary
# Each element of it is a list of matrices resulting from validate_n_replicates number of replicates. 
# Since by default validate_n_replicates = 1. It is simply a list of a single pd.DataFrame
model.W_simul_grid[(0.1, 0.0001)][0]

,Sig1,Sig2,Sig3,Sig4,Sig5,Sig6,Sig7,Sig8,Sig9
A[C>A]A,2.657707e-04,1.452666e-03,0.010708,0.006911,0.004682,0.020600,3.321905e-02,0.040214,0.014530
A[C>A]C,9.452294e-04,1.192093e-07,0.009539,0.007659,0.008384,0.009585,1.167049e-02,0.041452,0.014156
A[C>A]G,5.930639e-04,1.652816e-04,0.001700,0.001232,0.002287,0.001710,6.197097e-04,0.003563,0.002202
A[C>A]T,1.192093e-07,7.070524e-04,0.011359,0.003844,0.000977,0.017304,1.466100e-02,0.041854,0.009000
C[C>A]A,2.141418e-03,1.801688e-04,0.011159,0.004843,0.003654,0.007225,5.051520e-02,0.037024,0.018774
...,...,...,...,...,...,...,...,...,...
G[T>G]T,1.192093e-07,1.656614e-04,0.049638,0.000090,0.001331,0.004049,5.102910e-04,0.003932,0.002713
T[T>G]A,1.981760e-05,1.525191e-04,0.002140,0.001711,0.009379,0.004937,6.627864e-05,0.001652,0.007568
T[T>G]C,1.258739e-04,3.832309e-05,0.005367,0.004049,0.006041,0.000167,2.701699e-03,0.002442,0.007228
T[T>G]G,2.326039e-04,1.192093e-07,0.001563,0.005856,0.009599,0.000958,1.192093e-07,0.002156,0.011102


In [18]:
model.H_simul_grid[(0.1, 0.0001)][0]
# You can also get X_simul from: model.X_simul_grid

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
Sig1,61.222123,196.566393,60.221840,154.274465,376.930652,595.204195,82.169063,88.644645,168.918753,26251.916825,...,476.955538,232.399945,610.075867,1525.452446,253.431737,93.291473,2161.240831,2965.648534,67.466063,2811.576702
Sig2,36.124814,107.818527,558.318720,0.243384,162.744121,1248.394324,0.000000,20.972856,127.544647,31097.703902,...,358.978360,757.910533,117.308923,536.035624,1740.588246,1233.875333,4147.711664,3595.560820,543.944929,5818.526013
Sig3,40.007558,46.207022,298.276118,38.287649,30.738138,29.900673,22.182170,22.215629,24.491119,0.000000,...,61.347835,211.533152,127.997128,75.481892,330.237748,321.591422,79.265618,173.013459,218.481211,665.160626
Sig4,535.678848,931.619523,872.564038,609.778710,722.499407,600.972725,445.227211,435.459587,879.219725,0.000000,...,405.540665,782.906221,536.995531,1291.673842,616.996822,696.250288,1085.473050,2048.519560,495.191831,534.191967
Sig5,441.460571,251.935444,931.652955,318.759758,262.814747,334.560279,261.958164,487.029479,426.368043,0.000000,...,316.446170,735.216639,680.563074,451.848233,923.319942,673.940081,180.782696,687.274384,1182.111394,1159.740259
Sig6,435.939822,347.340000,1197.515511,264.987480,248.411981,303.193289,179.752677,410.800806,419.925136,0.000000,...,395.539996,1067.732639,578.021898,438.614670,1352.889273,1207.815049,206.981420,1303.441415,850.146275,2420.294064
Sig7,32.073651,186.413964,405.729467,654.602698,263.761988,245.911677,248.819981,31.103572,136.591717,0.000000,...,491.656636,236.706918,417.754932,298.541761,663.362300,349.738996,1016.563297,475.805994,251.658612,401.164213
Sig8,348.132127,206.879849,2623.807609,60.731731,214.378539,254.801325,96.770931,91.639393,340.393950,0.000000,...,144.441446,2330.436358,384.415601,324.786951,1451.189155,2144.761914,357.055341,1126.229107,2118.699094,5476.603175
Sig9,9.192080,200.939102,3870.523028,2.444068,235.786794,389.713314,127.129874,185.379923,176.766362,0.000000,...,121.957171,1990.836610,322.483047,0.000000,3507.203420,3492.046030,316.937700,1291.873665,2020.768528,3704.216450


We can then do a 1d search for refitting

In [19]:
model.assign_grid(W_catalog, 
                  thresh_match_grid=np.array([0.1]), 
                  thresh_refit_grid=np.array([0.001, 0.01, 0.1, 1, 5]), 
                  thresh_new_sig=0.0)

/Users/hujin/GitHub/MuSiCal/musical/denovo.py:1056: UserWarning: self.assign_grid has been previously called. This call will replace the previous results.
  UserWarning)


In [20]:
model.validate_grid()

Extracting signatures for n_components = 9..................
n_components = 9, replicate 0 finished.
Time elapsed: 3.09 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.8 seconds.


...

/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.61 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


In [21]:
# Best parameter:
print(model.best_grid_point)
print(model.thresh_match)
print(model.thresh_refit)

(0.1, 0.01)
0.1
0.01


In [22]:
# Assigned signatures
print(model.sigs_assigned)

['SBS1' 'SBS2' 'SBS3' 'SBS5' 'SBS8' 'SBS10c' 'SBS13' 'SBS17a' 'SBS17b'
 'SBS18' 'SBS30' 'SBS39' 'SBS44' 'SBS98']


In [23]:
# W distance on the grid:
model.W_cos_dist_mean_grid

{(0.1, 0.001): 0.06953240605001429,
 (0.1, 0.01): 0.08347453009461665,
 (0.1, 0.1): 0.09393536061006384,
 (0.1, 1.0): 0.22251759596936913,
 (0.1, 5.0): 0.3059923826652277}

In [24]:
# H distance on the grid:
model.H_frobenius_dist_mean_grid

{(0.1, 0.001): 23938.43244994405,
 (0.1, 0.01): 22370.883909568962,
 (0.1, 0.1): 40427.165634976765,
 (0.1, 1.0): 49645.61291306273,
 (0.1, 5.0): 70140.5659209773}

In [25]:
# Final result
model.W_s

,SBS1,SBS2,SBS3,SBS5,SBS8,SBS10c,SBS13,SBS17a,SBS17b,SBS18,SBS30,SBS39,SBS44,SBS98
Type,,,,,,,,,,,,,,
A[C>A]A,8.861572e-04,5.800168e-07,0.020808,0.011998,4.409822e-02,0.004331,1.819920e-03,2.070261e-03,0.000608,0.051534,0.001800,0.011702,7.681946e-18,0.013372
A[C>A]C,2.280405e-03,1.480043e-04,0.016507,0.009438,4.779807e-02,0.014830,7.209682e-04,9.181159e-04,0.000129,0.015810,0.000506,0.007151,1.500380e-04,0.010144
A[C>A]G,1.770314e-04,5.230151e-05,0.001751,0.001850,4.619813e-03,0.000657,2.639884e-04,4.760601e-05,0.000058,0.002432,0.000091,0.002670,9.162321e-07,0.002156
A[C>A]T,1.280227e-03,9.780282e-05,0.012205,0.006609,4.699810e-02,0.013128,3.479847e-04,6.180780e-05,0.000456,0.021414,0.000556,0.007401,5.781464e-03,0.012239
C[C>A]A,3.120554e-04,2.080060e-04,0.022509,0.007429,4.009838e-02,0.017433,1.399938e-03,2.950373e-04,0.000271,0.074049,0.000768,0.010101,2.840719e-05,0.007782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G[T>G]T,1.460259e-05,2.230064e-16,0.005832,0.002350,1.289948e-03,0.007474,2.889873e-06,7.240914e-03,0.113992,0.002141,0.000160,0.005941,2.800709e-03,0.004398
T[T>G]A,2.230396e-16,1.670048e-05,0.007253,0.005219,2.219910e-16,0.008731,1.839919e-04,2.220280e-16,0.000012,0.000696,0.000118,0.008391,7.681946e-18,0.008400
T[T>G]C,5.510978e-05,7.040203e-05,0.006283,0.006559,1.139954e-03,0.000952,2.219902e-16,1.160146e-04,0.008749,0.002101,0.000096,0.005081,7.681946e-18,0.005467


In [26]:
# Final result
model.H_s

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
SBS1,226.994246,458.283461,347.496279,365.112602,377.149132,343.742364,245.887644,196.075825,378.008021,0.000000,...,253.430546,407.943223,314.361840,567.117720,524.123515,0.000000,671.187169,1033.144238,248.647137,0.000000
SBS2,0.000000,139.110768,0.000000,0.000000,307.108138,815.481446,0.000000,0.000000,129.031854,30658.220252,...,429.294433,340.011037,479.218127,1379.396229,722.678265,0.000000,2883.245117,3366.025248,0.000000,3810.290462
SBS3,0.000000,0.000000,0.000000,0.000000,0.000000,958.256599,0.000000,0.000000,0.000000,0.000000,...,0.000000,2981.718312,0.000000,0.000000,8178.925538,0.000000,0.000000,0.000000,0.000000,0.000000
SBS5,1329.441507,1418.071715,3988.767555,844.815093,1257.565132,932.374459,992.770858,1499.329022,1676.061814,0.000000,...,1267.377688,2048.729055,1680.282209,2068.655330,0.000000,4266.087824,1783.238832,4584.165058,3194.896898,7295.079892
SBS8,333.748067,323.932345,3074.186805,0.000000,376.419428,0.000000,0.000000,0.000000,399.313290,0.000000,...,0.000000,1887.483229,0.000000,0.000000,0.000000,2548.969744,0.000000,1534.018590,2190.478982,6102.990124
SBS10c,0.000000,0.000000,0.000000,195.571256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SBS13,0.000000,80.561390,0.000000,0.000000,159.626043,914.061310,0.000000,0.000000,93.742496,24119.541257,...,297.231639,577.629788,152.123187,463.060384,1102.812446,803.459765,3092.896401,2851.865386,0.000000,4630.723175
SBS17a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SBS17b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SBS18,0.000000,0.000000,0.000000,371.558387,0.000000,0.000000,190.592384,0.000000,0.000000,0.000000,...,396.238220,0.000000,491.246196,365.811196,0.000000,0.000000,958.489685,0.000000,0.000000,0.000000
